# INF367 Mandatory 2

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import MinMaxScaler
from qiskit.circuit.library import UnitaryGate
from collections import Counter, defaultdict
from qiskit import QuantumCircuit, transpile
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [ ]:
SEED = 367

## Data exploration and pre-processing

In [ ]:
X, y = load_iris(return_X_y=True)
X_train, X_rest, y_train, y_rest = train_test_split(X, y, train_size=.7, random_state=SEED)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, random_state=SEED)
print("Training size: ", len(X_train))
print("Validation size: ", len(X_val))
print("Test size: ", len(X_test))

In [ ]:
print("Features shape: ",X_train.shape)
print("Target shape: ",y_train.shape)
print(f"Feature value range: {np.min(X_train)} : {np.max(X_train)}")
print("Target values: ", Counter(y_train))

In [ ]:
scaler = MinMaxScaler(feature_range=(0,np.pi))
X_train = scaler.fit_transform(X_train)

### Helper functions

In [ ]:
def output_mapping(binary_counts, n_classes):
    """
    Maps binary count output to n classes
    """
    mapping = {}
    class_preds = defaultdict(int)
    for binary in binary_counts:
        b_number = int(binary, base=2)
        c = b_number % n_classes
        mapping.update({binary:c})
        
    for binary, count in binary_counts.items():
        class_preds[mapping[binary]] += count
    
    return class_preds

In [ ]:
BACKEND = AerSimulator()
def measure_circuit(qc: QuantumCircuit, shots=1000):
    pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
    isa_circuit = pm.run(qc)
    result = BACKEND.run(isa_circuit, shots=shots).result()
    return result.get_counts()

## QNN-circuits

In [ ]:
# Qircuit 1
def make_circuit1(features, parameters):
    qc_1 = QuantumCircuit(4, 4)
    for i in range(4):
        qc_1.rx(qubit=i, theta=features[i])
    qc_1.barrier()

    #qc_1_unitary_V2 = UnitaryGate([[1,0], [0,1]])
    params = [Parameter(f"{i}") for i in range(9)]
    qc_1.append(Custom_UnitaryGate1([params[0], params[1]]), [3,2])
    qc_1.append(Custom_UnitaryGate1([params[0], params[1]]), [1,0])
    qc_1.barrier()

    qc_1.measure(qubit=2, cbit=2)
    qc_1.append(Custom_UnitaryGateV([params[2], params[3], params[4]]).control(1), [2,3])

    qc_1.measure(qubit=0, cbit=0)
    qc_1.append(Custom_UnitaryGateV([params[2], params[3], params[4]]).control(1), [0,1])
    
    qc_1.barrier()
    qc_1.append(Custom_UnitaryGate2([params[5], params[6], params[7], params[8]]), [3,1])
    qc_1.measure(qubit=1, cbit=1)
    qc_1.measure(qubit=3, cbit=3)

    qc_1 = qc_1.assign_parameters(parameters)
    return qc_1

def Custom_UnitaryGate1(parameters):
    qc = QuantumCircuit(2)
    qc.cx(0,1)
    qc.rz(parameters[0], 1)
    qc.ry(parameters[1], 0)
    qc.cx(0,1)
    return qc.to_gate(label="U1")

def Custom_UnitaryGate2(parameters):
    qc = QuantumCircuit(2)
    qc.cx(0,1)
    qc.rx(parameters[0], 1)
    qc.rx(parameters[1], 0)
    qc.rz(parameters[2], 0)
    qc.rz(parameters[3], 1)
    qc.cx(0,1)
    return qc.to_gate(label="U2")

def Custom_UnitaryGateV(parameters):
    qc = QuantumCircuit(1)
    qc.rx(parameters[0], 0)
    qc.ry(parameters[1], 0)
    qc.rz(parameters[2], 0)
    return qc.to_gate(label="V")

circ1 = make_circuit1([1.61567622, 1.83259571, 2.34288266, 3.00500167], [1,2,1,2,1,2,1,2,3])
circ1.decompose()
circ1.draw(reverse_bits=True, output="mpl")

In [ ]:
# Circuit 2

def circuit2(features, trainable_parameters, layers=2):
    input_size = len(features)
    qc = QuantumCircuit(input_size)
    for i in range(input_size):
        qc.rx(features[i], i)
    qc.barrier()

    for i in range(layers):
        for j in range(input_size):
            qc.ry(Parameter(f"phi{i}{j}"),j)
        for j in range(input_size-1, 0, -1):
            qc.cx(j, j-1)
        qc.barrier()

    qc = qc.assign_parameters(trainable_parameters)
    qc.measure_all()
    return qc
    
layers = 8
feature_size = 4
parameters = np.random.uniform(low=0, high=np.pi, size=(layers*feature_size,))
circ2 = circuit2([0, 1.64, 2.24, 3.0], parameters, layers)
circ2.draw("mpl", reverse_bits=True)

In [ ]:
results = measure_circuit(circ2, 1000)
output_mapping(results, 3)

In [ ]:
# Circuit 3

qc_3 = QuantumCircuit()

## Gradient Descent Function

## Training and Validation

## Test Performance